In [1]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests


In [2]:
# tool create 

@tool
def multiply(a: int, b:int) -> int:
    """Given 2 numbers a and b this tool return their product"""
    return a * b


In [3]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [4]:
multiply.name

'multiply'

In [5]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [6]:
multiply.description

'Given 2 numbers a and b this tool return their product'

In [10]:
MODEL = "qwen3"
llm = ChatOllama(
    model = MODEL
)

In [11]:
llm_with_tools = llm.bind_tools([multiply])

In [12]:
llm_with_tools.invoke("Hi How are you ?")

AIMessage(content='<think>\nOkay, the user said, "Hi How are you?" I need to respond appropriately. Let me check if there\'s any function I should use here. The available tool is the multiply function, which takes two integers and returns their product. But the user\'s message is just a greeting. They didn\'t ask for any calculation or use the multiply function. So, there\'s no need to call the multiply tool here. I should just reply with a friendly greeting. Let me make sure I\'m not missing any hidden request in their message. Nope, it\'s straightforward. Alright, respond normally.\n</think>\n\nHello! I\'m just a tool, but I\'m here and ready to help. How can I assist you today? 😊', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-10-05T05:46:13.2540307Z', 'done': True, 'done_reason': 'stop', 'total_duration': 24691580900, 'load_duration': 124480400, 'prompt_eval_count': 158, 'prompt_eval_duration': 799415400, 'eval_count': 150, 'eval_duration': 23765989

In [18]:
query = "can you multiply 3 with 10"
messages = [HumanMessage(query)]
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={})]

In [19]:
response = llm_with_tools.invoke(messages)

In [20]:
messages.append(response)

In [21]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content="<think>\nOkay, the user is asking to multiply 3 by 10. Let me check the tools available. There's a function called multiply that takes two integers, a and b. The parameters are required, so I need to use that function. The arguments should be a=3 and b=10. I'll format the tool call accordingly.\n</think>\n\n", additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-10-05T06:09:04.9224083Z', 'done': True, 'done_reason': 'stop', 'total_duration': 37014704600, 'load_duration': 5692912400, 'prompt_eval_count': 162, 'prompt_eval_duration': 13670344100, 'eval_count': 101, 'eval_duration': 17649538000, 'model_name': 'qwen3'}, id='run--24ed941e-a401-45c2-9b02-08d39dba69c4-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': '186efce2-ba98-4ce8-aa05-d6e4592b5b01', 'type': 'tool_call'}], usage_metadata={'input_tokens': 162, 'output_tokens': 

In [15]:
response.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'e81c6b96-4674-4a65-8888-8d7e51c186af',
 'type': 'tool_call'}

In [22]:
tool_result = multiply.invoke(response.tool_calls[0])

In [23]:
messages.append(tool_result)
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content="<think>\nOkay, the user is asking to multiply 3 by 10. Let me check the tools available. There's a function called multiply that takes two integers, a and b. The parameters are required, so I need to use that function. The arguments should be a=3 and b=10. I'll format the tool call accordingly.\n</think>\n\n", additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-10-05T06:09:04.9224083Z', 'done': True, 'done_reason': 'stop', 'total_duration': 37014704600, 'load_duration': 5692912400, 'prompt_eval_count': 162, 'prompt_eval_duration': 13670344100, 'eval_count': 101, 'eval_duration': 17649538000, 'model_name': 'qwen3'}, id='run--24ed941e-a401-45c2-9b02-08d39dba69c4-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': '186efce2-ba98-4ce8-aa05-d6e4592b5b01', 'type': 'tool_call'}], usage_metadata={'input_tokens': 162, 'output_tokens': 

In [24]:
llm_with_tools.invoke(messages)

AIMessage(content='<think>\n</think>\n\nThe product of 3 and 10 is 30.', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-10-05T06:10:34.9151033Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5622414900, 'load_duration': 106858100, 'prompt_eval_count': 253, 'prompt_eval_duration': 1477977600, 'eval_count': 19, 'eval_duration': 4000298600, 'model_name': 'qwen3'}, id='run--27964ce0-b5ff-4965-817f-6ef0787fc5dd-0', usage_metadata={'input_tokens': 253, 'output_tokens': 19, 'total_tokens': 272})

In [38]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [39]:
# tool create

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency factor between a given base currency and a target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/0de80b201608232b6797dd48/pair/{base_currency}/{target_currency}"

    response = requests.get(url=url)
    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Given a currency conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate




In [40]:
get_conversion_factor.invoke({"base_currency": "USD", "target_currency":"INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1759622401,
 'time_last_update_utc': 'Sun, 05 Oct 2025 00:00:01 +0000',
 'time_next_update_unix': 1759708801,
 'time_next_update_utc': 'Mon, 06 Oct 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.7925}

In [41]:
convert.invoke({"base_currency_value": 10, "conversion_rate": 88.7925})


887.9250000000001

In [42]:
llm = ChatOllama(
    model=MODEL
)

In [43]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [55]:
messages = [HumanMessage("What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd")]

In [56]:
ai_message = llm_with_tools.invoke(messages)

In [57]:
messages.append(ai_message)

In [58]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': '1db21aae-ccc9-4749-93f4-1a9ceb677cfb',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': '052f428b-5e54-4aaa-baeb-dfde7375447c',
  'type': 'tool_call'}]

In [59]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='<think>\nOkay, let\'s tackle the user\'s question. They want two things: the conversion factor between INR and USD, and then converting 10 INR to USD using that factor.\n\nFirst, I need to check if there\'s a function to get the conversion factor. The tools provided include \'get_conversion_factor\' which takes base_currency and target_currency. So I should call that with INR and USD. \n\nOnce I have the factor, the next step is to use the \'convert\' function. The \'convert\' function requires the base_currency_value, which in this case is 10 INR. But wait, the \'convert\' function\'s description says it calculates the target currency value from the base. So if the conversion factor is, say, 0.012 (meaning 1 INR = 0.012 USD), then 10 INR would be 10 * 0.012 = 0.12 USD. \n\nBut I need to make sure th

In [60]:
import json
for tool_call in ai_message.tool_calls:
    # execute the first tool and get the value of conversion rate
    if tool_call["name"] == "get_conversion_factor":
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # fetch this conversion rate
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        # append this tool message to message list
        messages.append(tool_message1)
    
    if tool_call["name"] == "convert":
        # fetch the current arg
        tool_call["args"]["conversion_rate"] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [61]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='<think>\nOkay, let\'s tackle the user\'s question. They want two things: the conversion factor between INR and USD, and then converting 10 INR to USD using that factor.\n\nFirst, I need to check if there\'s a function to get the conversion factor. The tools provided include \'get_conversion_factor\' which takes base_currency and target_currency. So I should call that with INR and USD. \n\nOnce I have the factor, the next step is to use the \'convert\' function. The \'convert\' function requires the base_currency_value, which in this case is 10 INR. But wait, the \'convert\' function\'s description says it calculates the target currency value from the base. So if the conversion factor is, say, 0.012 (meaning 1 INR = 0.012 USD), then 10 INR would be 10 * 0.012 = 0.12 USD. \n\nBut I need to make sure th

In [62]:
llm_with_tools.invoke(messages).content

"<think>\nOkay, let me process this step by step. The user asked for two things: the conversion factor between INR and USD, and then converting 10 INR to USD.\n\nFirst, I called the get_conversion_factor function with INR as the base and USD as the target. The response came back with a conversion rate of 0.01126. That means 1 INR equals 0.01126 USD. \n\nNext, the user wanted to convert 10 INR to USD. I used the convert function with the base_currency_value set to 10. The tool response gave 0.11259999999999999, which is approximately 0.1126 USD. \n\nSo, putting it all together, the conversion factor is 0.01126, and 10 INR is about 0.1126 USD. I should present this clearly, making sure to mention both parts of the user's request and round the numbers for readability.\n</think>\n\nThe conversion factor between INR (Indian Rupee) and USD (US Dollar) is **0.01126**, meaning 1 INR equals 0.01126 USD.\n\nUsing this rate, **10 INR** converts to:  \n**0.1126 USD** (rounded to 4 decimal places).